In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import pandas as pd
import numpy as np
import torch
import collections, re
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
!pip3 install nltk
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from keras.layers import  CuDNNLSTM 

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
clean_data=pd.read_csv("/content/drive/My Drive/NLP project/final_clean_data.csv",encoding='latin-1') #you need to change the path

In [5]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

train_indices, test_indices, y_train, y_test = train_test_split(np.arange(len(clean_data)), clean_data['fraudulent'],
                                                    stratify=clean_data['fraudulent'], 
                                                    test_size=0.3,random_state=0)
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(np.array(train_indices).reshape(-1, 1), clean_data.iloc[train_indices]['fraudulent'])
X_resampled=X_resampled.flatten()

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [0]:
non_text=clean_data.drop(['title','job_id','company_profile','description','fraudulent','Ambiguous Range','missing_salary','missing_stat','ambiguous_salary'], axis=1)
non_text2=clean_data.drop(['title','job_id','company_profile','description','fraudulent',], axis=1)

In [0]:
max_fatures = 3000
clean_data['description']=clean_data['description'].astype(str)
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(clean_data['description'].values)
X1 = tokenizer.texts_to_sequences(clean_data['description'].values)
X1 = pad_sequences(X1)

max_fatures = 3000
clean_data['company_profile']=clean_data['description'].astype(str)
tokenizer1 = Tokenizer(num_words=max_fatures, split=' ')
tokenizer1.fit_on_texts(clean_data['company_profile'].values)
X2 = tokenizer.texts_to_sequences(clean_data['company_profile'].values)
X2 = pad_sequences(X1)

combined=np.concatenate((X1, X2), axis=1)
combined=np.concatenate((combined, non_text), axis=1)
combined2=np.concatenate((combined, non_text2), axis=1)

In [9]:
from sklearn.metrics import classification_report, confusion_matrix,f1_score,accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=3000)
classifier.fit(combined[X_resampled], y_resampled)
y_pred = classifier.predict(combined[test_indices])
print(f1_score(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(precision_score(y_test,y_pred))
print(recall_score(y_test,y_pred))

0.2410291130670278
0.7910141685309471
0.14626129827444537
0.6846153846153846


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [10]:
from sklearn.metrics import classification_report, confusion_matrix,f1_score,accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=3000)
classifier.fit(combined2[X_resampled], y_resampled)
y_pred = classifier.predict(combined2[test_indices])
print(f1_score(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(precision_score(y_test,y_pred))
print(recall_score(y_test,y_pred))

0.24769992922859163
0.801826994780015
0.15177797051170858
0.6730769230769231


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [11]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state=0,max_depth=20,n_estimators=1000)
classifier.fit(combined[X_resampled], y_resampled)
y_pred = classifier.predict(combined[test_indices])
print(f1_score(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(precision_score(y_test,y_pred))
print(recall_score(y_test,y_pred))

0.6351706036745406
0.9740865026099925
1.0
0.4653846153846154


In [12]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state=0,max_depth=20,n_estimators=1000)
classifier.fit(combined2[X_resampled], y_resampled)
y_pred = classifier.predict(combined2[test_indices])
print(f1_score(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(precision_score(y_test,y_pred))
print(recall_score(y_test,y_pred))

0.6528497409326425
0.9750186428038777
1.0
0.4846153846153846
